In [1]:
#Utility

def solve_linear_system(eqns, variables, param):
    A = matrix([[eqn.coefficient(var) for var in variables] for eqn in eqns])
    b = matrix([sum([-eqn.coefficient(par)*par for par in param]) for eqn in eqns]).T
    sol = A.adjugate()*b
    return [sol[i,0] for i in range(len(variables))] + [det(A)*par for par in param]


def plane_coefficients(plane):
    return [plane.coefficient(vr) for vr in plane.parent().gens()[0:4]]


def get_plane_containing_two_incident_lines(line1, line2):
    P1, P2 = line1.points
    PL2 = line2.points
    for point in PL2:
        #check if point is on line1
        if matrix([P1, P2, point]).rank() > 2:
            #take determinant of 4x4 matrix to get equation
            return matrix([P1, P2, point, list(line1.P.gens()[0:4])]).det()
        

def find_first_nonzero_element(l):
    for i in range(len(l)):
        if l[i] != 0:
            return (l[i], i)
    return None

def are_vectors_proportional(elem1, elem2):
    e1 = find_first_nonzero_element(elem1)
    e2 = find_first_nonzero_element(elem2)
    if e1[1] != e2[1]:
        return False
    lin_comb = e2[0]*vector(elem1) - e1[0]*vector(elem2)
    if lin_comb == vector([0 for i in range(len(elem1))]):
        return True
    else:
        return False
    
def are_matrices_equal(m1, m2):
    elem1 = m1.list()
    elem2 = m2.list()
    return are_vectors_proportional(elem1, elem2)


def is_in_group(g, G):
    for el in G:
        if are_matrices_equal(g, el):
            return True
    return False

def is_abelian(G):
    for m1 in G:
        for m2 in G:
            if not are_matrices_equal(m1*m2, m2*m1):
                return False
    return True 

In [2]:
#Finding lines with class

def find_all_lines_on_cubic_surface(line, surface):
    all_lines = [line]
    #First line was already done
    for i in range(3):
        all_lines += get_new_lines_on_cubic_surface(all_lines[i], surface, all_lines)
        if len(all_lines) == 27:
            break
    return all_lines


#Starting from the given lines, it tries to find new lines
def get_new_lines_on_cubic_surface(line, surface, starting_lines):
    possible_new_lines = []
    possible_new_lines += find_lines_when_first_parameter_is_nonzero(line, surface)
    possible_new_lines += find_lines_when_first_parameter_is_zero(line, surface)
    new_lines = []
    for l in possible_new_lines:
        flag = True
        for l2 in starting_lines:
            if are_vectors_proportional(l.plucker, l2.plucker):
                flag = False
                break
        if flag:
            new_lines.append(l)
    return new_lines

def find_lines_when_first_parameter_is_nonzero(line, surface):
    P, conic, plane = define_equations(line, surface, 0)
    vr = P.gens()[0:4]
    conic_discriminant = find_conic_discriminant(conic)
    new_lines = []
    for eqn in [factor[0] for factor in conic_discriminant]:
        # exclude case when the eliminated variable is zero
        if eqn.variables() == (P.gens()[-2],):
            continue
        # avoid factors without any parameter
        if P.gens()[-2] and P.gens()[-1] not in eqn.variables():
            continue
        first_param_root = eqn.coefficient(P.gens()[-1])
        second_param_root = -eqn.coefficient(P.gens()[-2])
        factors_of_conic = conic.subs({P.gens()[-2]: first_param_root, P.gens()[-1]: second_param_root}).factor()
        factors = [factor[0] for factor in factors_of_conic if [v in factor[0].variables() for v in vr] != [False for i in range(4)]]
        non_param_plane = plane.subs({P.gens()[-2]: first_param_root, P.gens()[-1]: second_param_root})
        new_lines.append(Line([non_param_plane, factors[0]]))
        new_lines.append(Line([non_param_plane, factors[1]]))
    return new_lines
        
def find_lines_when_first_parameter_is_zero(line, surface):
    P, conic, plane = define_equations(line, surface, 1)
    vr = P.gens()[0:4]
    conic_discriminant = find_conic_discriminant(conic)
    new_lines = []
    for eqn in [factor[0] for factor in conic_discriminant]:
        if eqn.variables() == (P.gens()[-2],):
            factors_of_conic = conic.subs({P.gens()[-2]: 0, P.gens()[-1]: 1}).factor()
            non_param_plane = plane.subs({P.gens()[-2]: 0, P.gens()[-1]: 1})
            factors = [f[0] for f in factors_of_conic if [v in f[0].variables() for v in vr] != [False for i in range(4)]]
            new_lines.append(Line([non_param_plane, factors[0]]))
            new_lines.append(Line([non_param_plane, factors[1]]))
        else:
            continue
    return new_lines


def define_equations(line, surface, k):
    P = surface.parent()
    plane = line.planes[0] * P.gens()[-2] + line.planes[1] * P.gens()[-1]
    var = line.planes[k].variables()[0]
    param = [vr for vr in P.gens()[0:4] if vr != var]
    sol = solve_linear_system([plane], [var], param)
    sost = {param[i-1]: sol[i] for i in range(1,4)}
    sost[var]= sol[0]
    for fact in surface.subs(sost).factor():
        for deg in fact[0].degrees()[0:4]:
            if deg>1:
                return P, fact[0], plane
        
def find_conic_discriminant(conic):
    # do not consider l, m as variables for the conic
    P = conic.parent()
    v = [variable for variable in conic.variables() if variable in P.gens()[0:4]]
    conic_discriminant = matrix([
        [conic.coefficient(v[0] ^ 2),
         (1 / 2) * conic.coefficient(v[0]).coefficient(v[1]),
         (1 / 2) * conic.coefficient(v[0]).coefficient(v[2])],
        [(1 / 2) * conic.coefficient(v[1]).coefficient(v[0]),
         conic.coefficient(v[1] ^ 2),
         (1 / 2) * conic.coefficient(v[1]).coefficient(v[2])],
        [(1 / 2) * conic.coefficient(v[2]).coefficient(v[0]),
         (1 / 2) * conic.coefficient(v[2]).coefficient(v[1]),
         conic.coefficient(v[2] ^ 2)]
    ])
    return conic_discriminant.det().factor()

In [3]:
#Classify lines

def classify_lines_on_cubic_surface(all_lines):
    E = find_E(all_lines)
    G = find_G(E, [line for line in all_lines if line not in E])
    F = find_F(E, [line for line in all_lines if line not in E and line not in G])
    lines_dict = dict(E1=E[0], E2=E[1], E3=E[2], E4=E[3], E5=E[4], E6=E[5],
                  G1=G[0], G2=G[1], G3=G[2], G4=G[3], G5=G[4], G6=G[5],
                  F12=F[0][1], F13=F[0][2], F14=F[0][3], F15=F[0][4], F16=F[0][5],
                  F23=F[1][2], F24=F[1][3], F25=F[1][4], F26=F[1][5], F34=F[2][3], 
                  F35=F[2][4], F36=F[2][5], F45=F[3][4], F46=F[3][5], F56=F[4][5])
    return lines_dict


def find_E(all_lines):
    P = all_lines[0].P
    E1 = Line([y,z])
    E2 = Line([x,t])
    E3 = Line([x-y, z+t])
    G3 = Line([x-t, y-z])
    G4 = Line([x,y])
    E4=[]
    for line in all_lines:
        incidence_relations = [line.are_incident(e) for e in [E1, E2, E3, G4]]
        if incidence_relations == [False, False, False, False] and line.are_incident(G3):
            E4 = line
            break
    E = get_other_skew_lines([E1, E2, E3, E4], all_lines)
    return E


def get_other_skew_lines(skew_lines, all_lines):
    for line in all_lines:
        is_line_skew = true
        for e in skew_lines:
            if line.are_incident(e):
                is_line_skew = false
        if is_line_skew:       
            skew_lines.append(line)
        if len(skew_lines) == 6:
            break
    return skew_lines


def find_G(E, possible_lines):
    G = [0 for i in range(6)]
    for line in possible_lines:
        incidence_relations = [line.are_incident(e) for e in E]
        if incidence_relations.count(true) == 5:
            G[incidence_relations.index(false)] = line
            continue 
    return G
 
    
def find_F(E, possible_lines):
    F = [[None for i in range(6)] for j in range(6)]
    for line in possible_lines:
        incidence_relations = [line.are_incident(e) for e in E]
        first_index = incidence_relations.index(true)
        second_index = incidence_relations.index(true, first_index+1)
        F[first_index][second_index] = line
    return F

In [4]:
# L-sets

def find_5_lines_with_incidence_pattern(lines_dictionary):
    five_tuples = []
    keys = lines_dictionary.keys()
    for key1 in keys:
        for key2 in get_incident_keys(key1):
            for key3 in set(get_non_incident_keys(key1)).intersection(get_incident_keys(key2)):
                for key4 in set(get_incident_keys(key1)).intersection(
                    get_non_incident_keys(key2)).intersection(
                    get_incident_keys(key3)):
                    
                    for key5 in set(get_non_incident_keys(key1)).intersection(
                        get_incident_keys(key2)).intersection(
                        get_non_incident_keys(key3)).intersection(
                        get_non_incident_keys(key4)):
                        
                        five_tuples.append([key1, key2, key3, key4, key5])
    return five_tuples


def get_non_incident_keys(key):
    not_incident_lines = []
    if key[0] == 'E':
        index = int(key[1])
        not_incident_lines += ['E' + str(i + 1) for i in range(6) if i + 1 != index]
        not_incident_lines.append('G' + str(index))
        not_incident_lines += ['F' + str(i + 1) + str(j + 1) for i in range(6) for j in range(i + 1, 6)
                               if i + 1 != index and j + 1 != index]
    elif key[0] == 'G':
        index = int(key[1])
        not_incident_lines += ['G' + str(i + 1) for i in range(6) if i + 1 != index]
        not_incident_lines.append('E' + str(index))
        not_incident_lines += ['F' + str(i + 1) + str(j + 1) for i in range(6) for j in range(i + 1, 6)
                               if i + 1 != index and j + 1 != index]
    else:
        index1 = int(key[1])
        index2 = int(key[2])
        not_incident_lines += ['E' + str(i + 1) for i in range(6) if i + 1 != index1 and i + 1 != index2]
        not_incident_lines += ['G' + str(i + 1) for i in range(6) if i + 1 != index1 and i + 1 != index2]
        not_incident_lines += ['F' + str(i + 1) + str(j + 1) for i in range(6) for j in range(i + 1, 6)
                               if len(set([i + 1, j + 1, index1, index2])) < 4]
        not_incident_lines.remove('F' + str(index1) + str(index2))
    return not_incident_lines


def get_incident_keys(key):
    incident_lines = []
    if key[0] == 'E':
        index = int(key[1])
        incident_lines += ['G' + str(i + 1) for i in range(6) if i + 1 != index]
        incident_lines += ['F' + str(i + 1) + str(j + 1) for i in range(6) for j in range(i + 1, 6)
                           if i + 1 == index or j + 1 == index]
    elif key[0] == 'G':
        index = int(key[1])
        incident_lines += ['E' + str(i + 1) for i in range(6) if i + 1 != index]
        incident_lines += ['F' + str(i + 1) + str(j + 1) for i in range(6) for j in range(i + 1, 6)
                           if i + 1 == index or j + 1 == index]
    else:
        index1 = int(key[1])
        index2 = int(key[2])
        incident_lines += ['E' + str(i + 1) for i in range(6) if i + 1 == index1 or i + 1 == index2]
        incident_lines += ['G' + str(i + 1) for i in range(6) if i + 1 == index1 or i + 1 == index2]
        incident_lines += ['F' + str(i + 1) + str(j + 1) for i in range(6) for j in range(i + 1, 6)
                           if len(set([i + 1, j + 1, index1, index2])) == 4]
    return incident_lines


#from the classification as E, F, G returns the actual lines in plucker coordinates
def get_L_set_in_plucker(classified_lines, L_set):
    return tuple(map(lambda uu: classified_lines[uu], L_set))

        
# A = l1 ∩ l2, B = l1 ∩ l4, C = l3 ∩ l4, D = l2 ∩ l3, E = l2 ∩ l5
# P := l4 ∩ <l2 + l5>, Q := <P + D> ∩ l5.
def get_five_points_in_general_position(L_set):
    A = L_set[0].intersection_point(L_set[1])
    B = L_set[0].intersection_point(L_set[3])
    C = L_set[2].intersection_point(L_set[3])
    D = L_set[1].intersection_point(L_set[2])
    E = L_set[1].intersection_point(L_set[4])
    
    plane_l2_l5 = get_plane_containing_two_incident_lines(L_set[1], L_set[4])
    plane_coeff = plane_coefficients(plane_l2_l5)
    a = vector(plane_coeff[1:4])
    pl = L_set[3].plucker
    d = vector(pl[0:3])
    m = vector([pl[5], -pl[4], pl[3]])
    P = [a.dot_product(d)]+list(a.cross_product(m) - plane_coeff[0]*d) 
    
    plane_l3_l4 = get_plane_containing_two_incident_lines(L_set[2], L_set[3])
    line_P_D = Line([plane_l2_l5, plane_l3_l4])
    Q = line_P_D.intersection_point(L_set[4])
    return A,B,C,E,Q


def find_projectivity(base_five_points, L_set2):
    P = L_set2[0].P
    SS = PolynomialRing(P, 21, 'v')
    vrs = SS.gens()[:16]
    dummy_vars = SS.gens()[-5:]
    points2 = get_five_points_in_general_position(L_set2)
    M = matrix([[var for var in SS.gens()[i:i+4]] for i in range(0, 15, 4)])
    system = matrix([base_five_points[i] for i in range(5)])*M
    b = diagonal_matrix(dummy_vars)*matrix(points2[i] for i in range(5))
    eqn = system - b
    sol = solve_linear_system(eqn.list(), SS.gens()[0:20], [SS.gens()[-1]])
    sost = {SS.gens()[i] : sol[i] for i in range(16)}
    proj = M.subs(sost).subs({SS.gens()[-1] : 1}).change_ring(P)
    proj = (proj/gcd(proj.list()) ).list()
    proj = matrix([[P(el) for el in proj[i:i+4]] for i in range(0, 15, 4)])
    return proj/gcd(gcd(proj.coefficients()[i].coefficients()) for i in range(len(proj.coefficients())))


def find_all_projectivities(classified_lines, L_set):
    L_set_base = get_L_set_in_plucker(classified_lines, ['E1', 'G4', 'E2', 'G3', 'E3'])
    base_five_points = get_five_points_in_general_position(L_set_base)
    L2 = get_L_set_in_plucker(classified_lines, L_set)
    M = find_projectivity(base_five_points, L2)
    return M  

def find_proj_parallel(args):
    return find_all_projectivities(*args)

def find_all_proj_parallel(cl_lines, all_L_sets):
    all_param = [(cl_lines, L_set) for L_set in all_L_sets]
    pool = mp.Pool(mp.cpu_count()-1)
    return pool.map(find_proj_parallel, all_param)


#find the projectiviy which sends the base L_set to another L_set
def find_projectivity2(base_five_points, L_set2):
    P = L_set2[0].P
    SS = PolynomialRing(QQ, 21, 'v')
    vrs = SS.gens()[:16]
    dummy_vars = SS.gens()[-5:]
    points1 = [list(map(SS, list(map(QQ, point)))) for point in base_five_points]
    points2 = [list(map(SS, list(map(QQ, point)))) for point in get_five_points_in_general_position(L_set2)]
    A1, B1, C1, E1, Q1 = (vector(p) for p in points1)
    A2, B2, C2, E2, Q2 = (vector(p) for p in points2)
    M = matrix([[var for var in SS.gens()[i:i+4]] for i in range(0, 15, 4)])
    system = matrix([A1, B1, C1, E1, Q1])*M
    b = diagonal_matrix(dummy_vars)*matrix([A2, B2, C2, E2, Q2])
    eqn = system - b
    sys_ideal = SS.ideal(eqn.list())
    param_sol = list(map(lambda uu: sys_ideal.reduce(uu), vrs))
    sol_dict = {vrs[i]: param_sol[i] for i in range(16)}
    proj_transf = M.subs(sol_dict).subs({v:1 for v in dummy_vars})
    return matrix([list(map(P, list(map(QQ, proj_transf[i])))) for i in range(4)])


def find_all_projectivities2(classified_lines, all_L_sets):
    L_set_base = get_L_set_in_plucker(classified_lines, ['E1', 'G4', 'E2', 'G3', 'E3'])
    M=[]
    base_five_points = get_five_points_in_general_position(L_set_base)
    for a in all_L_sets:
        L2 = get_L_set_in_plucker(classified_lines, a)
        M.append(find_projectivity(base_five_points, L2))
    return M 

In [5]:
#Cubics


def are_same_cubic(f1, f2):
    P = f1.parent()
    vrs = P.gens()[0:4]
    if len(f1.coefficients()) != len(f2.coefficients()):
        return False
    monom = [vrs[0]^i*vrs[1]^j*vrs[2]^k*vrs[3]^l for l in range(4) for k in range(4) for j in range(4) for i in range(4) if i+j+k+l == 3]
    c1 = [f1.coefficient(var) for var in monom]
    c2 = [f2.coefficient(var) for var in monom]
    return are_vectors_proportional(c1, c2)


def find_simmetries(cubic, all_projectivities):
    P = cubic.parent()
    vrs = P.gens()[0:4]
    simmetries = []
    for M in all_projectivities:
        change_coord = vector(vrs)*M
        sost = {vrs[i] : change_coord[i] for i in range(4)}
        if are_same_cubic(cubic, cubic.subs(sost)):
            simmetries.append(M)
    return simmetries

In [6]:
class Line():
    def __init__(self, planes):
        self.P = planes[0].parent()
        self.planes = planes
        self.points = self.get_two_points_on_line()
        self.plucker = self.get_plucker_coordinates(self.points)

    
    def __str__(self):
        return str(self.plucker) + ' ' + str(self.planes) 

        
    def get_plucker_coordinates(self, points):
        P1, P2 = points
        coordinates = vector(matrix([P1, P2]).minors(2))
        coordinates = vector(map(lambda uu: P(uu/gcd(coordinates)), coordinates))        
        #make it so that the first nonzero coordinate is positive
        if coordinates[coordinates.nonzero_positions()[0]] < 0:
            return -coordinates
        return coordinates
    
    def get_two_points_on_line(self):
        vr = self.P.gens()
        M = matrix([plane_coefficients(plane) for plane in self.planes])
        minors = M.minors(2)
        index = next(i[0] for i in enumerate(minors) if i[1] != 0)
        possible_columns = {0:(0,1), 1:(0,2), 2:(0,3), 3:(1,2), 4:(1,3), 5:(2,3)}
        columns = possible_columns.get(index)
        other_columns = tuple({0,1,2,3} - set(columns))
        sol = solve_linear_system(self.planes, [vr[columns[0]], vr[columns[1]]], [vr[other_columns[0]], vr[other_columns[1]]])
        sost = {vr[columns[0]]: sol[0], vr[columns[1]]:sol[1], vr[other_columns[0]]:sol[2], vr[other_columns[1]]:sol[3]}
        point1_sost = {vr[other_columns[0]]:0, vr[other_columns[1]]:1}
        point2_sost = {vr[other_columns[0]]:1, vr[other_columns[1]]:0}
        
        not_ordered_vars = [vr[columns[0]], vr[columns[1]], vr[other_columns[0]], vr[other_columns[1]]]   
        ordered_vars = []
        for i in range(4):
            for el in not_ordered_vars:
                if el == vr[i]:
                    ordered_vars.append(el)
                    continue
        point1 = vector([el.subs(sost).subs(point1_sost) for el in ordered_vars])
        point2 = vector([el.subs(sost).subs(point2_sost) for el in ordered_vars])
        point1 = vector(map(lambda uu: P(uu/gcd(point1)), point1))
        point2 = vector(map(lambda uu: P(uu/gcd(point2)), point2))
        return point1, point2
    

    def are_incident(self, other_line):
        d = [self.plucker[i]*other_line.plucker[5-i] for i in range(6)]
        return d[0]-d[1]+d[2]+d[3]-d[4]+d[5] == 0

    
    def intersection_point(self, other_line):
        vrs = self.P.gens()
        plane1, plane2 = self.planes
        plane3, plane4 = other_line.planes
        if matrix([plane_coefficients(plane) for plane in [plane1, plane2, plane3]]).rank() == 3:
            sol = solve_linear_system([plane1, plane2, plane3], vrs[0:3], [vrs[3]])
            return vector([s.subs({vrs[3]:1}) for s in sol])
        else:
            sol = solve_linear_system([plane1, plane2, plane4], vrs[0:3], [vrs[3]])
            return vector([s.subs({vrs[3]:1}) for s in sol])